In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder\
                    .appName('AnalyzingLondonCrimeData')\
                    .getOrCreate()

In [4]:
data = spark.read\
            .format('csv')\
            .option('header', 'true')\
            .load('..\datasets\GettingStartedSpark2\london_crime_by_lsoa.csv')

In [5]:
data.printSchema()

root
 |-- lsoa_code: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- major_category: string (nullable = true)
 |-- minor_category: string (nullable = true)
 |-- value: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)



In [6]:
data.count()

13490604

In [7]:
data.limit(5).show()

+---------+----------+--------------------+--------------------+-----+----+-----+
|lsoa_code|   borough|      major_category|      minor_category|value|year|month|
+---------+----------+--------------------+--------------------+-----+----+-----+
|E01001116|   Croydon|            Burglary|Burglary in Other...|    0|2016|   11|
|E01001646| Greenwich|Violence Against ...|      Other violence|    0|2016|   11|
|E01000677|   Bromley|Violence Against ...|      Other violence|    0|2015|    5|
|E01003774| Redbridge|            Burglary|Burglary in Other...|    0|2016|    3|
|E01004563|Wandsworth|             Robbery|   Personal Property|    0|2008|    6|
+---------+----------+--------------------+--------------------+-----+----+-----+



In [8]:
data.dropna()

DataFrame[lsoa_code: string, borough: string, major_category: string, minor_category: string, value: string, year: string, month: string]

In [9]:
data = data.drop('lsoa_code')
data.show(5)

+----------+--------------------+--------------------+-----+----+-----+
|   borough|      major_category|      minor_category|value|year|month|
+----------+--------------------+--------------------+-----+----+-----+
|   Croydon|            Burglary|Burglary in Other...|    0|2016|   11|
| Greenwich|Violence Against ...|      Other violence|    0|2016|   11|
|   Bromley|Violence Against ...|      Other violence|    0|2015|    5|
| Redbridge|            Burglary|Burglary in Other...|    0|2016|    3|
|Wandsworth|             Robbery|   Personal Property|    0|2008|    6|
+----------+--------------------+--------------------+-----+----+-----+
only showing top 5 rows



In [10]:
all_boroughs = data.select('borough').distinct()
all_boroughs.show()

+--------------------+
|             borough|
+--------------------+
|             Croydon|
|          Wandsworth|
|              Bexley|
|             Lambeth|
|Barking and Dagenham|
|              Camden|
|           Greenwich|
|              Newham|
|       Tower Hamlets|
|            Hounslow|
|              Barnet|
|              Harrow|
|Kensington and Ch...|
|           Islington|
|               Brent|
|            Haringey|
|             Bromley|
|              Merton|
|         Westminster|
|             Hackney|
+--------------------+
only showing top 20 rows



In [11]:
hackney_data = data.filter(data['borough'] == 'Hackney')
hackney_data.show(5)

+-------+--------------------+--------------------+-----+----+-----+
|borough|      major_category|      minor_category|value|year|month|
+-------+--------------------+--------------------+-----+----+-----+
|Hackney|     Criminal Damage|Criminal Damage T...|    0|2011|    6|
|Hackney|Violence Against ...|          Harassment|    1|2013|    2|
|Hackney|     Criminal Damage|Other Criminal Da...|    0|2011|    7|
|Hackney|Violence Against ...|        Wounding/GBH|    0|2013|   12|
|Hackney|  Theft and Handling|  Other Theft Person|    0|2016|    8|
+-------+--------------------+--------------------+-----+----+-----+
only showing top 5 rows



In [12]:
data_2015_2016 = data.filter(data['year'].isin(['2015', '2016']))
data_2015_2016.sample(fraction=0.1).show()

+--------------------+--------------------+--------------------+-----+----+-----+
|             borough|      major_category|      minor_category|value|year|month|
+--------------------+--------------------+--------------------+-----+----+-----+
|            Haringey|Violence Against ...|        Wounding/GBH|    0|2015|   12|
|            Haringey|     Criminal Damage|Criminal Damage T...|    0|2016|   12|
|           Islington|  Theft and Handling|Theft/Taking Of M...|    0|2016|    6|
|Kensington and Ch...|Other Notifiable ...|    Other Notifiable|    0|2015|    5|
|       Tower Hamlets|             Robbery|   Personal Property|    0|2016|   10|
|             Lambeth|     Criminal Damage|Criminal Damage T...|    0|2016|    7|
|              Ealing|  Theft and Handling|Theft/Taking Of M...|    0|2016|    1|
|              Barnet|  Theft and Handling|Handling Stolen G...|    0|2016|    4|
|              Newham|  Theft and Handling|         Other Theft|    0|2015|    6|
|          Hilli

In [13]:
borough_crime_count = data.groupBy('borough').count()
borough_crime_count.show(5)

+--------------------+------+
|             borough| count|
+--------------------+------+
|             Croydon|602100|
|          Wandsworth|498636|
|              Bexley|385668|
|             Lambeth|519048|
|Barking and Dagenham|311040|
+--------------------+------+
only showing top 5 rows



In [14]:
# get all crimes in a borough, which where solved
borough_conviction_sum = data.groupBy('borough').agg({'value':'sum'}).withColumnRenamed('sum(value)', 'convictions')
borough_conviction_sum.show(5)

+--------------------+-----------+
|             borough|convictions|
+--------------------+-----------+
|             Croydon|   260294.0|
|          Wandsworth|   204741.0|
|              Bexley|   114136.0|
|             Lambeth|   292178.0|
|Barking and Dagenham|   149447.0|
+--------------------+-----------+
only showing top 5 rows



In [15]:
# all convictions without grouping
convictions_sum = borough_conviction_sum.agg({'convictions':'sum'})
convictions_sum.show()

+----------------+
|sum(convictions)|
+----------------+
|       6447758.0|
+----------------+



In [16]:
total_convictions = convictions_sum.collect()[0][0]

In [17]:
import pyspark.sql.functions as func

In [18]:
borough_percentage_contribution = borough_conviction_sum.withColumn('% contribution', 
                                                                    func.round(borough_conviction_sum.convictions/total_convictions * 100, 2))

borough_percentage_contribution.printSchema()

root
 |-- borough: string (nullable = true)
 |-- convictions: double (nullable = true)
 |-- % contribution: double (nullable = true)



In [19]:
borough_percentage_contribution.orderBy(borough_percentage_contribution[2].desc()).show(10)

+-------------+-----------+--------------+
|      borough|convictions|% contribution|
+-------------+-----------+--------------+
|  Westminster|   455028.0|          7.06|
|      Lambeth|   292178.0|          4.53|
|    Southwark|   278809.0|          4.32|
|       Camden|   275147.0|          4.27|
|       Newham|   262024.0|          4.06|
|      Croydon|   260294.0|          4.04|
|       Ealing|   251562.0|           3.9|
|    Islington|   230286.0|          3.57|
|Tower Hamlets|   228613.0|          3.55|
|        Brent|   227551.0|          3.53|
+-------------+-----------+--------------+
only showing top 10 rows



In [1]:
crimes_category = data.groupBy('major_category').agg({'value':'sum'}).withColumnRenamed('sum(value)', 'convictions')

NameError: name 'data' is not defined